<font size=6  color= 'white' > <b>[LEPL1507] - PROJET P4</b> <br>
<br><br>
Airson Alexis<br>
Beniffou Ibrahim <br>
Henneaux Lucas <br>
Lemaire Antoine <br>
Smith Marielle <br>
Canon Théo<br>
<div style="text-align: right"> </div>

<br><br>
</font>


This is the version maintaining the essential parts of code to run

Easier to add new features 


<font size=6  color= 'white' > <b> Phase 1 : retrieving data </b> <br>

The aim of this first part is to get arrays of images 

In [ ]:
# CELL 1.1 : accessing to data 
# Needs [ CELL 0 : import ]

# defines variables 

#-----------------------------------------------------------------------------

path_training = 'Retrieving_data\BelgiumTSC_Training\Training'
path_testing = 'Retrieving_data\BelgiumTSC_Testing\Testing'

nbr_class = 62 # len(next(walk(path_training))[1])  counting the number of classes 
print('number of classes : ', nbr_class)

#-----------------------------------------------------------------------------

# resol is the format for the images that we want for convenience

resol = (50,50) # Square image only ! 
print("Resolution of images : ", resol)

In [ ]:
from Retrieving_data.retrieving_data import accessing

# These are iterators 
# See fct accessing in codes of Phase 1 
training_set = accessing(path_training, resol)
test_set = accessing(path_testing, resol) 

In [ ]:
from Retrieving_data.retrieving_data import store

# These are arrays
# See fct store in codes of Phase 1 
X_train, y_train = store(training_set)
X_test, y_test = store(test_set)

<font size=6  color= 'white' > <b> Phase 2 : visualisation of the dataset </b> <br>

The aim of this part is to visualise the initial datasets 

In [ ]:
from Visualisation.Visu import number
from copy import deepcopy

initial_nbr_train = number(nbr_class, deepcopy(y_train))
initial_nbr_test = number(nbr_class, deepcopy(y_test))

In [ ]:
from Visualisation.Visu import graphs
from copy import deepcopy

graphs(nbr_class, deepcopy(y_train), deepcopy(y_test), 'y_train', 'y_test')

<font size=6  color= 'white' > <b> Phase 3 : randomization of the dataset </b> <br>

The aim of this part is to avoid overfitting by applying transformations on the images 

In [ ]:
from keras import Sequential
from keras.layers import RandomZoom, RandomRotation, RandomCrop, RandomContrast
from tensorflow import get_logger
from copy import deepcopy
from numpy import array

# This cell is about data augmentation 
# We'll randomize our set and do our augmentation with this (cells later)
# We choose to work with the second method of augmentation  

#-----------------------------------------------------------------------------

get_logger().setLevel('ERROR')

data_augmentation = Sequential() 

data_augmentation.add(RandomZoom(height_factor=(-0.3, -0.2),width_factor=(-0.3, -0.2)))
data_augmentation.add(RandomRotation((-0.02, 0.02)))
data_augmentation.add(RandomCrop(height=resol[0],width=resol[1]))
data_augmentation.add(RandomContrast(0.2))

#-----------------------------------------------------------------------------

# We apply the augmentation on our datasets 
augmented_image_train = array(data_augmentation(deepcopy(X_train)))

#-----------------------------------------------------------------------------

print("Shape of the randomized training set : ", augmented_image_train.shape)

In [ ]:
from random import randint 
from matplotlib.pyplot import figure, imshow

# A little test to see the result of the augmentation 

#-----------------------------------------------------------------------------

index = randint(0,len(X_train))
figure()
imshow(X_train[index])
figure()
imshow(augmented_image_train[index])

<font size=6  color= 'white' > <b> Phase 4 : augmentation of the dataset </b> <br>

The aim of this part is to change the sizes of the initial datasets


In [ ]:
from augmentation.aug import first 
from copy import deepcopy

X_train_first, y_train_first = first(nbr_class, deepcopy(initial_nbr_train), deepcopy(initial_nbr_test), deepcopy(augmented_image_train), deepcopy(y_train), resol)

In [ ]:
from Visualisation.Visu import graphs, number

nbr_train_first = number(nbr_class, y_train_first)
graphs(nbr_class, y_train_first.copy(), y_test.copy(), 'y_train_first', 'y_test')

In [ ]:
from augmentation.aug import second 
from copy import deepcopy

X_train_second, y_train_second = second(nbr_class, deepcopy(initial_nbr_train), deepcopy(augmented_image_train), deepcopy(y_train), resol, max(initial_nbr_train))

In [ ]:
from Visualisation.Visu import graphs, number

nbr_train_second = number(nbr_class, y_train_second)
graphs(nbr_class, y_train_second.copy(), y_test.copy(), 'y_train_second', 'y_test')

In [ ]:
from keras.utils import to_categorical
from copy import deepcopy

# We put our results to categorical 

#-----------------------------------------------------------------------------

y_test_tc = to_categorical(deepcopy(y_test), nbr_class)
y_train_tc = to_categorical(deepcopy(y_train), nbr_class)

print("Shape of y_train without augmentation : ", y_train_tc.shape)
print("Shape of y_test without augmentation : ", y_test_tc.shape)

#-----------------------------------------------------------------------------

y_train_first_tc = to_categorical(y_train_first, nbr_class)


print("Shape of y_train with first method : ", y_train_first_tc.shape)
print("Shape of y_test with first method : ", y_test_tc.shape)

#-----------------------------------------------------------------------------

y_train_second_tc = to_categorical(y_train_second, nbr_class)

print("Shape of y_train with second method : ", y_train_second_tc.shape)
print("Shape of y_test with second method : ", y_test_tc.shape)

<font size=6  color= 'white' > <b> Phase 5 : Construction of the model </b> <br>

The aim of this part is to construct a model using CNN

You can find an optimizer for hyperparameters with the codes of this part

In [ ]:
from Model.model import construct_model
from copy import deepcopy

model_simple = construct_model(nbr_class, deepcopy(X_train))

model_first = construct_model(nbr_class, deepcopy(X_train_first))

model_second = construct_model(nbr_class, deepcopy(X_train_second))

model_second.summary() # Same model so same parameters for the 3 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from copy import deepcopy

# We train here the model with the first method of augmentation 

eps = 3 # The number of time we want the model to train on the entire training_set

#-----------------------------------------------------------------------------

mycallbacks = [EarlyStopping(monitor='val_loss', patience=5)]

#-----------------------------------------------------------------------------

validation_X = deepcopy(X_test)
validation_y = deepcopy(y_test_tc)

#-----------------------------------------------------------------------------

anc_second = model_second.fit(X_train_second, y_train_second_tc,validation_data=(validation_X,validation_y), epochs=eps, callbacks=mycallbacks)

#-----------------------------------------------------------------------------

#added aug.flow to (X_train, y_train) to do data augmentation
# validation_data=(X_test_new_first, y_test_new_first)
# validation_data=(X_test, y_test)

# talk : aug.flow ? 
# talk : validation data strange 

<font size=6  color= 'white' > <b> Phase 6 : Analysis of performances </b> <br>

The aim of this part is to analyse the results of the model based on the testing set available 

In [ ]:
label_names = open("dict.csv").read().strip().split("\n")[0:]
label_names = [l.split(",")[1] for l in label_names]

In [ ]:
from Performances.perf import perf 

perf(anc_second)

In [ ]:
from Performances.perf import ratio_kaggle

ratio_kaggle(y_test_tc, X_test, model_second)

In [ ]:
from Performances.perf import network

from warnings import filterwarnings

filterwarnings('ignore')

network(model_second, y_test_tc, X_test, label_names, nbr_class)

<font size=6  color= 'white' > <b> Phase 7 : Visualisation of the first kaggle challenge </b> <br>

The aim of this part is to visualise the dataset of the first challenge 

In [ ]:
from Visualisation.Visu import to_jpeg

folder_dir = "challenge_1/eval_kaggle1"
to_jpeg(folder_dir)

<font size=6  color= 'white' > <b> Phase 8 : Generating csv for kaggle </b> <br>

The aim of this part is to predict on the kaggle dataset and generate a csv 

In [ ]:
from CSV.CSV import store_2

images, names = store_2('challenge_1/eval_kaggle1', resol)

In [ ]:
from csv import DictReader

# We will store the data in dict.csv in a dict 

data = {}

with open('dict.csv', 'r') as f:
    d_reader = DictReader(f, fieldnames=["num", "sign"])

    #get fieldnames from DictReader object and store in list
    for row in d_reader:
        data[row['num']] = row['sign']

In [ ]:
from matplotlib.pyplot import figure, title, imshow 
from numpy import uint8

# Here's a little test to visualise some results 
# We then print the image with num and the sign predicted as a title 

predictions = model_second.predict(images).argmax(axis=1) 

for i in range(20): 
    figure(figsize = (10,10))
    imshow((images[i] * 255).astype(uint8))
    sign = data[str(predictions[i])]
    title(str(predictions[i]) + " : " + str(sign))



In [ ]:
# We generate the csv file for kaggle 

from CSV.CSV import write 

write(names, predictions, 'kaggle_1')

<font size=6  color= 'white' > <b> Part 2 : Cropping </b> <br>

<font size=6  color= 'white' > <b> Phase 1 : Retrieving images </b> <br>

The aim of this part is to retrieve the names of the images to be cropped 

In [ ]:
from os.path import exists
from os import makedirs, system

#-----------------------------------------------------------------------------

path = "eval_kaggle2"
dest = "result_croppped_eval_2"

#-----------------------------------------------------------------------------

# Creating required directories 

if exists(dest):
        system('rm -r dest')

if not exists(dest):
        makedirs(dest)

if not exists(path):
        makedirs(path)

#-----------------------------------------------------------------------------

In [ ]:
from Retrieving_data.retrieving_data import accessing_2

#-----------------------------------------------------------------------------

directory = path

names = accessing_2(directory)

print("The number of images to be cropped is : ", len(names))

<font size=6  color= 'white' > <b> Phase 2 : Detecting contours </b> <br>

The aim of this part is to detect contours on grayscaled images

In [ ]:
from Contours.contours import detection 

contours = detection(names)

In [ ]:
# A little test to show the result 

import cv2 
import matplotlib.pyplot as plt 

for i in range (6) : 

    im = cv2.imread(names[i])
    img = cv2.drawContours(im, contours[i], -1, (0,255,0), 3)
    plt.figure(figsize=(5,5))
    plt.imshow(img)

<font size=6  color= 'white' > <b> Phase 3 : Detecting forms </b> <br>

The aim of this part is to detect forms in the contours

In [ ]:
from Contours.contours import classify, forms

forms = {} 

for i in range (len(contours)) : 

    result = classify(contours[i])

    forms[i] = (result[0], result[1], result[2], result[3])

In [ ]:
print(max(forms[5][3], key=(forms[5][3]).get))
largest = max(forms[5][3])